## Step 0

In [1]:
# ЛОГИЧЕСКИЙ БЛОК: imports + reproducibility + GLOBAL config
# ИСПОЛНЕНИЕ БЛОКА:

import os, math, random
import numpy as np
import pandas as pd
from pathlib import Path

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from sklearn.preprocessing import RobustScaler
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, roc_auc_score

def seed_everything(seed=1234):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything(100)

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("DEVICE:", DEVICE)

# -------------------------------
# GLOBAL CONFIG (всё тут)
# -------------------------------
CFG = {
    # data
    "freq": "5min",
    "data_dir": Path("../dataset"),
    # NEW: holdout final test split (по времени, на sample-space)
    "final_test_frac": 0.10,

    "book_levels": 15,         # сколько уровней стакана грузим
    "top_levels": 5,           # DI_L0..DI_L4
    "near_levels": 5,          # near=0..4, far=5..14

    # walk-forward windows (в sample-space)
    "train_min_frac": 0.50,
    "val_window_frac": 0.10,
    "test_window_frac": 0.10,
    "step_window_frac": 0.10,

    # scaling
    "max_abs_feat": 10.0,

    # correlations
    "corr_windows": [6, 12, 24, 48, 84],  # 30m,1h,2h,4h,7h
    "edges": [("ADA","BTC"), ("ADA","ETH"), ("ETH","BTC")],

    # triple-barrier (labels)
    "tb_horizon": 1*12,       # 1h     # нужен для sample_t (чтобы TB-exit не вылезал за конец)
    "lookback": 7*12,
    "tb_pt_mult": 1.2,
    "tb_sl_mult": 1.1,
    "tb_min_barrier": 0.001,
    "tb_max_barrier": 0.006,

    # training (общие)
    "batch_size": 64,
    "epochs": 30,
    "lr": 2e-4,
    "weight_decay": 1e-3,
    "grad_clip": 1.0,
    "dropout": 0.3,
    "hidden": 64,
    "gnn_layers": 2,

    # --- old Attn params kept for compatibility: now used as ATTENTION dims ---
    # Attn_hidden -> d_model (размер темпорального представления)
    # Attn_layers -> n_layers (кол-во attention/transformer layers, если режим их использует)
    "Attn_hidden": 64,
    "Attn_layers": 1,

    "use_amp": True,

    # trading eval
    "cost_bps": 2.0,

    # confidence thresholds (для PnL по порогу)
    "thr_trade_grid": [0.50, 0.55, 0.60, 0.65, 0.70],
    "thr_dir_grid":   [0.50, 0.55, 0.60, 0.65, 0.70],

    # ---- PnL proxy during DIR training (grid selector)
    "proxy_min_trades": 20,

    # -------------------------------
    # NEW: attention-based temporal encoder config
    # -------------------------------
    # ДВА режима (по твоему требованию):
    # 1) "xformer_cls"  -> TransformerEncoder + [CLS] token, берём CLS как summary
    # 2) "attn_pool"    -> learnable query + MultiHeadAttention pooling (быстрый и стабильный)
    "temporal_mode": "xformer_cls",   # "xformer_cls" | "attn_pool"
    "attn_heads": 4,                  # будет автоматически приведено к делителю d_model
    "attn_ff_mult": 4,                # FFN dim = ff_mult * d_model (для xformer_cls)
    "attn_dropout": None,             # None -> использовать CFG["dropout"]
    "attn_causal": False,             # опционально для xformer (обычно False, т.к. окно past-only)
    "attn_use_pos_emb": True,         # learned positional embeddings
}

ASSETS = ["ADA", "BTC", "ETH"]
ASSET2IDX = {a:i for i,a in enumerate(ASSETS)}
TARGET_ASSET = "ETH"
TARGET_NODE = ASSET2IDX[TARGET_ASSET]

EDGES = CFG["edges"]
EDGE_INDEX = torch.tensor([[ASSET2IDX[s], ASSET2IDX[t]] for (s,t) in EDGES], dtype=torch.long)  # [E,2]
print("EDGE_INDEX:", EDGE_INDEX.tolist())

print("TEMPORAL:", CFG["temporal_mode"], "| heads=", CFG["attn_heads"], "| d_model=", CFG["Attn_hidden"])


DEVICE: cpu
EDGE_INDEX: [[0, 1], [0, 2], [2, 1]]
TEMPORAL: xformer_cls | heads= 4 | d_model= 64


## 1. load data + basic returns

In [2]:
# ЛОГИЧЕСКИЙ БЛОК: load data + log returns (без target) + все уровни стакана
# ИСПОЛНЕНИЕ БЛОКА:

def load_asset(asset: str, freq: str, data_dir: Path, book_levels: int, part = [0,100]) -> pd.DataFrame:
    path = data_dir / f"{asset}_{freq}.csv"
    df = pd.read_csv(path)
    df = df.iloc[int(len(df)*part[0]/100) : int(len(df)*part[1]/100)]
    df["timestamp"] = pd.to_datetime(df["system_time"]).dt.round("min")
    df = df.sort_values("timestamp").set_index("timestamp")

    bid_cols = [f"bids_notional_{i}" for i in range(book_levels)]
    ask_cols = [f"asks_notional_{i}" for i in range(book_levels)]

    needed = ["midpoint", "spread", "buys", "sells"] + bid_cols + ask_cols
    missing = [c for c in needed if c not in df.columns]
    if missing:
        raise ValueError(f"{asset}: missing columns in CSV: {missing[:10]}{'...' if len(missing) > 10 else ''}")

    return df[needed]


def load_all_assets() -> pd.DataFrame:
    freq = CFG["freq"]
    data_dir = CFG["data_dir"]
    book_levels = CFG["book_levels"]

    def rename_asset_cols(df_one: pd.DataFrame, asset: str) -> pd.DataFrame:
        rename_map = {
            "midpoint": asset,
            "buys": f"buys_{asset}",
            "sells": f"sells_{asset}",
            "spread": f"spread_{asset}",
        }
        for i in range(book_levels):
            rename_map[f"bids_notional_{i}"] = f"bids_vol_{asset}_{i}"
            rename_map[f"asks_notional_{i}"] = f"asks_vol_{asset}_{i}"
        return df_one.rename(columns=rename_map)

    df_ADA = rename_asset_cols(load_asset("ADA", freq, data_dir, book_levels, part = [0, 80]), "ADA")
    df_BTC = rename_asset_cols(load_asset("BTC", freq, data_dir, book_levels, part = [0, 80]), "BTC")
    df_ETH = rename_asset_cols(load_asset("ETH", freq, data_dir, book_levels, part = [0, 80]), "ETH")

    df = df_ADA.join(df_BTC).join(df_ETH)
    df = df.reset_index()  # timestamp column remains
    return df


df = load_all_assets()
T = len(df)

# log returns
for a in ASSETS:
    df[f"lr_{a}"] = np.log(df[a]).diff().fillna(0.0)

print("Loaded df:", df.shape)
print("Example columns:", df.columns[:25].tolist())


Loaded df: (2693, 106)
Example columns: ['timestamp', 'ADA', 'spread_ADA', 'buys_ADA', 'sells_ADA', 'bids_vol_ADA_0', 'bids_vol_ADA_1', 'bids_vol_ADA_2', 'bids_vol_ADA_3', 'bids_vol_ADA_4', 'bids_vol_ADA_5', 'bids_vol_ADA_6', 'bids_vol_ADA_7', 'bids_vol_ADA_8', 'bids_vol_ADA_9', 'bids_vol_ADA_10', 'bids_vol_ADA_11', 'bids_vol_ADA_12', 'bids_vol_ADA_13', 'bids_vol_ADA_14', 'asks_vol_ADA_0', 'asks_vol_ADA_1', 'asks_vol_ADA_2', 'asks_vol_ADA_3', 'asks_vol_ADA_4']


## 2. multi-window correlations → edge features (T,E,W)

In [3]:
# ЛОГИЧЕСКИЙ БЛОК: multi-window correlations -> corr_array (T,E,W)
# ИСПОЛНЕНИЕ БЛОКА:

candidate_windows = CFG["corr_windows"]
edges = EDGES

n_w = len(candidate_windows)
n_edges = len(edges)
T = len(df)

corr_array = np.zeros((T, n_edges, n_w), dtype=np.float32)

for wi, w in enumerate(candidate_windows):
    r_ADA_BTC = df["lr_ADA"].rolling(w, min_periods=1).corr(df["lr_BTC"])
    r_ADA_ETH = df["lr_ADA"].rolling(w, min_periods=1).corr(df["lr_ETH"])
    r_ETH_BTC = df["lr_ETH"].rolling(w, min_periods=1).corr(df["lr_BTC"])

    corr_array[:, 0, wi] = np.nan_to_num(r_ADA_BTC)
    corr_array[:, 1, wi] = np.nan_to_num(r_ADA_ETH)
    corr_array[:, 2, wi] = np.nan_to_num(r_ETH_BTC)

print("corr_array shape:", corr_array.shape)  # (T,E,W)


corr_array shape: (2693, 3, 5)


## 3. triple-barrier → y_tb + exit_ret → two-stage labels

In [4]:
# ЛОГИЧЕСКИЙ БЛОК: triple-barrier labels -> y_tb + exit_ret + two-stage labels
# ИСПОЛНЕНИЕ БЛОКА:

def triple_barrier_labels_from_lr(
    lr: pd.Series,
    horizon: int,
    vol_window: int,
    pt_mult: float,
    sl_mult: float,
    min_barrier: float,
    max_barrier: float,
):
    """
    Returns:
      y_tb: {0=down, 1=flat/no-trade, 2=up}
      exit_ret: realized log-return to exit (tp/sl/timeout)
      exit_t: exit index
      thr: barrier per t
    No leakage: vol is shift(1).
    """
    lr = lr.astype(float).copy()
    T = len(lr)

    vol = lr.rolling(vol_window, min_periods=max(10, vol_window//10)).std().shift(1)
    thr = (vol * np.sqrt(horizon)).clip(lower=min_barrier, upper=max_barrier)

    y = np.ones(T, dtype=np.int64)
    exit_ret = np.zeros(T, dtype=np.float32)
    exit_t = np.arange(T, dtype=np.int64)

    lr_np = lr.fillna(0.0).to_numpy(dtype=np.float64)
    thr_np = thr.fillna(min_barrier).to_numpy(dtype=np.float64)

    for t in range(T - horizon - 1):
        up = pt_mult * thr_np[t]
        dn = -sl_mult * thr_np[t]

        cum = 0.0
        hit = 1
        et = t + horizon
        er = 0.0

        for dt in range(1, horizon + 1):
            cum += lr_np[t + dt]
            if cum >= up:
                hit = 2
                et = t + dt
                er = cum
                break
            if cum <= dn:
                hit = 0
                et = t + dt
                er = cum
                break

        if hit == 1:
            er = float(np.sum(lr_np[t+1:t+horizon+1]))
            et = t + horizon

        y[t] = hit
        exit_ret[t] = er
        exit_t[t] = et

    return y, exit_ret, exit_t, thr_np

# --- build TB on ETH ---
y_tb, exit_ret, exit_t, thr = triple_barrier_labels_from_lr(
    df["lr_ETH"],
    horizon=1*12, 
    vol_window=7*12,
    pt_mult=1.2,
    sl_mult=1.1,
    min_barrier=0.001,
    max_barrier=0.006,
)

# two-stage labels
y_trade = (y_tb != 1).astype(np.int64)      # 1=trade, 0=no-trade
y_dir   = (y_tb == 2).astype(np.int64)      # 1=up, 0=down (для trade-сэмплов)

print("TB dist [down,flat,up]:", np.bincount(y_tb, minlength=3))
print("Trade ratio:", y_trade.mean())


TB dist [down,flat,up]: [ 655 1311  727]
Trade ratio: 0.5131823245451169


## 4. build node tensor + edge tensor + sample_t

In [5]:
# ЛОГИЧЕСКИЙ БЛОК: build node features (T,N,F) + edge features (T,E,W) + sample_t
# ИСПОЛНЕНИЕ БЛОКА:

EPS = 1e-6

def safe_log1p(x: np.ndarray) -> np.ndarray:
    return np.log1p(np.maximum(x, 0.0))

def build_node_tensor(df: pd.DataFrame):
    """
    Фичи на asset:
      lr, spread,
      log_buys, log_sells, ofi,
      DI_15,
      DI_L0..DI_L4,
      near_ratio_bid, near_ratio_ask,
      di_near, di_far
    """
    feats = []
    feat_names = [
        "lr", "spread",
        "log_buys", "log_sells", "ofi",
        "DI_15",
        "DI_L0", "DI_L1", "DI_L2", "DI_L3", "DI_L4",
        "near_ratio_bid", "near_ratio_ask",
        "di_near", "di_far",
    ]

    book_levels = CFG["book_levels"]
    top_k = CFG["top_levels"]     # 5
    near_k = CFG["near_levels"]   # 5
    far_k = book_levels - near_k
    if far_k <= 0:
        raise ValueError("CFG['near_levels'] must be < CFG['book_levels']")

    for a in ASSETS:
        lr = df[f"lr_{a}"].values.astype(np.float32)
        spread = df[f"spread_{a}"].values.astype(np.float32)

        buys = df[f"buys_{a}"].values.astype(np.float32)
        sells = df[f"sells_{a}"].values.astype(np.float32)

        log_buys = safe_log1p(buys).astype(np.float32)
        log_sells = safe_log1p(sells).astype(np.float32)

        ofi = ((buys - sells) / (buys + sells + EPS)).astype(np.float32)

        # уровни стакана
        bids_lvls = np.stack([df[f"bids_vol_{a}_{i}"].values.astype(np.float32) for i in range(book_levels)], axis=1)  # (T,15)
        asks_lvls = np.stack([df[f"asks_vol_{a}_{i}"].values.astype(np.float32) for i in range(book_levels)], axis=1)  # (T,15)

        bid_sum_15 = bids_lvls.sum(axis=1)
        ask_sum_15 = asks_lvls.sum(axis=1)
        DI_15 = ((bid_sum_15 - ask_sum_15) / (bid_sum_15 + ask_sum_15 + EPS)).astype(np.float32)

        # DI_L0..DI_L4
        di_levels = []
        for i in range(top_k):
            b = bids_lvls[:, i]
            s = asks_lvls[:, i]
            di_levels.append(((b - s) / (b + s + EPS)).astype(np.float32))
        DI_L0_4 = np.stack(di_levels, axis=1)  # (T,5)

        # near vs far
        bid_near = bids_lvls[:, :near_k].sum(axis=1)
        ask_near = asks_lvls[:, :near_k].sum(axis=1)
        bid_far = bids_lvls[:, near_k:].sum(axis=1)
        ask_far = asks_lvls[:, near_k:].sum(axis=1)

        near_ratio_bid = (bid_near / (bid_far + EPS)).astype(np.float32)
        near_ratio_ask = (ask_near / (ask_far + EPS)).astype(np.float32)

        di_near = ((bid_near - ask_near) / (bid_near + ask_near + EPS)).astype(np.float32)
        di_far = ((bid_far - ask_far) / (bid_far + ask_far + EPS)).astype(np.float32)

        Xa = np.column_stack([
            lr, spread,
            log_buys, log_sells, ofi,
            DI_15,
            DI_L0_4[:, 0], DI_L0_4[:, 1], DI_L0_4[:, 2], DI_L0_4[:, 3], DI_L0_4[:, 4],
            near_ratio_bid, near_ratio_ask,
            di_near, di_far
        ]).astype(np.float32)

        feats.append(Xa)

    X = np.stack(feats, axis=1).astype(np.float32)  # (T,N,F)
    return X, feat_names


X_node_raw, node_feat_names = build_node_tensor(df)
edge_feat = np.nan_to_num(corr_array.astype(np.float32), nan=0.0, posinf=0.0, neginf=0.0)

T = len(df)
L = CFG["lookback"]
H = CFG["tb_horizon"]

# sample_t: чтобы можно было брать окно [t-L+1 ... t] и иметь будущий TB-exit без выхода за данные
t_min = L - 1
t_max = T - H - 2
sample_t = np.arange(t_min, t_max + 1)
n_samples = len(sample_t)

print("X_node_raw:", X_node_raw.shape, "edge_feat:", edge_feat.shape)
print("node_feat_names:", node_feat_names)
print("n_samples:", n_samples, "t range:", sample_t[0], sample_t[-1])


X_node_raw: (2693, 3, 15) edge_feat: (2693, 3, 5)
node_feat_names: ['lr', 'spread', 'log_buys', 'log_sells', 'ofi', 'DI_15', 'DI_L0', 'DI_L1', 'DI_L2', 'DI_L3', 'DI_L4', 'near_ratio_bid', 'near_ratio_ask', 'di_near', 'di_far']
n_samples: 2597 t range: 83 2679


## Train (folds) - Test split

In [6]:
# ЛОГИЧЕСКИЙ БЛОК: final holdout split (90% CV + 10% final test), time-ordered
# ИСПОЛНЕНИЕ БЛОКА:

def make_final_holdout_split(n_samples: int, final_test_frac: float):
    if not (0.0 < final_test_frac < 0.5):
        raise ValueError("final_test_frac should be in (0, 0.5)")

    n_final = max(1, int(round(final_test_frac * n_samples)))
    n_cv = n_samples - n_final
    if n_cv <= 10:
        raise ValueError("Too few samples left for CV after holdout split.")

    idx_cv = np.arange(0, n_cv, dtype=np.int64)
    idx_final = np.arange(n_cv, n_samples, dtype=np.int64)
    return idx_cv, idx_final, n_cv, n_final

idx_cv_all, idx_final_test, n_samples_cv, n_samples_final = make_final_holdout_split(
    n_samples=n_samples,
    final_test_frac=CFG["final_test_frac"],
)

print("Holdout split:")
print("  n_samples total:", n_samples)
print("  n_samples CV   :", n_samples_cv, f"({100*(n_samples_cv/n_samples):.1f}%)")
print("  n_samples FINAL:", n_samples_final, f"({100*(n_samples_final/n_samples):.1f}%)")
print("  CV range   :", idx_cv_all[0], idx_cv_all[-1])
print("  FINAL range:", idx_final_test[0], idx_final_test[-1])


Holdout split:
  n_samples total: 2597
  n_samples CV   : 2337 (90.0%)
  n_samples FINAL: 260 (10.0%)
  CV range   : 0 2336
  FINAL range: 2337 2596



## 5. walk-forward splits (с глобальными окнами)

In [7]:
# ЛОГИЧЕСКИЙ БЛОК: walk-forward splits (expanding train + fixed val/test) on CV-part only
# ИСПОЛНЕНИЕ БЛОКА:

def make_walk_forward_splits(n_samples: int,
                             train_min_frac: float,
                             val_window_frac: float,
                             test_window_frac: float,
                             step_window_frac: float):
    train_min = int(train_min_frac * n_samples)
    val_w  = max(1, int(val_window_frac * n_samples))
    test_w = max(1, int(test_window_frac * n_samples))
    step_w = max(1, int(step_window_frac * n_samples))

    splits = []
    start = train_min
    while True:
        tr_end = start
        va_end = tr_end + val_w
        te_end = va_end + test_w
        if te_end > n_samples:
            break

        idx_train = np.arange(0, tr_end, dtype=np.int64)
        idx_val   = np.arange(tr_end, va_end, dtype=np.int64)
        idx_test  = np.arange(va_end, te_end, dtype=np.int64)

        splits.append((idx_train, idx_val, idx_test))
        start += step_w

    return splits

# IMPORTANT: строим сплиты только на 90% (CV-part)
walk_splits = make_walk_forward_splits(
    n_samples=n_samples_cv,
    train_min_frac=CFG["train_min_frac"],
    val_window_frac=CFG["val_window_frac"],
    test_window_frac=CFG["test_window_frac"],
    step_window_frac=CFG["step_window_frac"],
)

print("n_folds:", len(walk_splits))
for i, (a, b, c) in enumerate(walk_splits):
    print(f" fold {i+1}: train {len(a)} | val {len(b)} | test {len(c)}")

print("\nFINAL HOLDOUT:")
print(" final_test size:", len(idx_final_test))


n_folds: 4
 fold 1: train 1168 | val 233 | test 233
 fold 2: train 1401 | val 233 | test 233
 fold 3: train 1634 | val 233 | test 233
 fold 4: train 1867 | val 233 | test 233

FINAL HOLDOUT:
 final_test size: 260


## 6. Dataset + scaling 

In [8]:
# ЛОГИЧЕСКИЙ БЛОК: Dataset + scaling (shared)
# ИСПОЛНЕНИЕ БЛОКА:

class LobGraphSequenceDataset2Stage(Dataset):
    """
    Возвращает (x_seq, e_seq, y_trade, y_dir, exit_ret)
    y_dir корректен только когда y_trade==1, но мы возвращаем всегда.
    """
    def __init__(self, X_node, E_feat, y_trade, y_dir, exit_ret, sample_t, indices, lookback):
        self.X_node = X_node
        self.E_feat = E_feat
        self.y_trade = y_trade
        self.y_dir = y_dir
        self.exit_ret = exit_ret
        self.sample_t = sample_t
        self.indices = indices
        self.L = lookback

    def __len__(self):
        return len(self.indices)

    def __getitem__(self, i):
        sidx = self.indices[i]
        t = self.sample_t[sidx]
        t0 = t - self.L + 1

        x_seq = self.X_node[t0:t+1]     # (L,N,F)
        e_seq = self.E_feat[t0:t+1]     # (L,E,W)

        yt = self.y_trade[t]
        yd = self.y_dir[t]
        er = self.exit_ret[t]

        return (
            torch.from_numpy(x_seq),
            torch.from_numpy(e_seq),
            torch.tensor(yt, dtype=torch.long),
            torch.tensor(yd, dtype=torch.long),
            torch.tensor(er, dtype=torch.float32),
        )

def collate_fn_2stage(batch):
    xs, es, yts, yds, ers = zip(*batch)
    return (
        torch.stack(xs, 0),   # (B,L,N,F)
        torch.stack(es, 0),   # (B,L,E,W)
        torch.stack(yts, 0),  # (B,)
        torch.stack(yds, 0),  # (B,)
        torch.stack(ers, 0),  # (B,)
    )

def fit_scale_nodes_train_only(X_node_raw, sample_t, idx_train, max_abs=10.0):
    """
    Fit scaler on all times up to last train sample time (без leakage).
    """
    last_train_t = sample_t[idx_train[-1]]
    train_time_mask = np.arange(0, last_train_t + 1)

    X_train_time = X_node_raw[train_time_mask]  # (Ttr,N,F)
    Ttr, N, Fdim = X_train_time.shape

    scaler = RobustScaler(with_centering=True, with_scaling=True, quantile_range=(5.0, 95.0))
    scaler.fit(X_train_time.reshape(-1, Fdim))

    X_scaled = scaler.transform(X_node_raw.reshape(-1, Fdim)).reshape(X_node_raw.shape).astype(np.float32)
    X_scaled = np.clip(X_scaled, -max_abs, max_abs).astype(np.float32)
    X_scaled = np.nan_to_num(X_scaled, nan=0.0, posinf=0.0, neginf=0.0).astype(np.float32)
    return X_scaled, scaler

def subset_trade_indices(indices, sample_t, y_trade):
    """
    indices в sample-space -> отфильтровать те, где y_trade[t]==1
    """
    tt = sample_t[indices]
    mask = (y_trade[tt] == 1)
    return indices[mask]


## 7.Model (один класс, n_classes=2) + EdgeGatedMP

In [9]:
# ЛОГИЧЕСКИЙ БЛОК: GNN + ATTENTION temporal classifier (drop-in вместо Attn)
# ИСПОЛНЕНИЕ БЛОКА:

class EdgeGatedMP(nn.Module):
    def __init__(self, in_dim, hidden, edge_dim, dropout=0.1):
        super().__init__()
        self.node_proj = nn.Linear(in_dim, hidden)
        self.ln0 = nn.LayerNorm(hidden)

        self.edge_mlp = nn.Sequential(
            nn.Linear(2*hidden + edge_dim, 2*hidden),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(2*hidden, hidden + 1)  # msg(hidden) + gate(1)
        )

        self.upd = nn.Sequential(
            nn.Linear(2*hidden, 2*hidden),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(2*hidden, hidden)
        )
        self.ln1 = nn.LayerNorm(hidden)
        self.dropout = nn.Dropout(dropout)

    def forward_once(self, x_t, edge_attr_t, edge_index):
        B, N, _ = x_t.shape
        E = edge_index.shape[0]

        h = self.ln0(self.node_proj(x_t))  # (B,N,H)
        h = torch.nan_to_num(h, nan=0.0, posinf=0.0, neginf=0.0)

        agg = torch.zeros((B, N, h.shape[-1]), device=h.device, dtype=h.dtype)

        for e in range(E):
            src = edge_index[e, 0].item()
            dst = edge_index[e, 1].item()
            h_src = h[:, src, :]
            h_dst = h[:, dst, :]
            ea = edge_attr_t[:, e, :]

            z = torch.cat([h_src, h_dst, ea], dim=-1)
            out = self.edge_mlp(z)
            msg = out[:, :-1]
            gate = torch.sigmoid(out[:, -1:])

            agg[:, dst, :] += msg * gate

        h2 = self.upd(torch.cat([h, agg], dim=-1))
        h2 = self.ln1(h + self.dropout(h2))
        h2 = torch.nan_to_num(h2, nan=0.0, posinf=0.0, neginf=0.0)
        return h2

    def forward(self, x_seq, e_seq, edge_index):
        B, L, N, Fin = x_seq.shape
        h_out = []
        for t in range(L):
            ht = self.forward_once(x_seq[:, t, :, :], e_seq[:, t, :, :], edge_index)
            h_out.append(ht)
        return torch.stack(h_out, dim=1)  # (B,L,N,H)


def _greatest_divisor_leq(x: int, k: int) -> int:
    k = max(1, min(k, x))
    for d in range(k, 0, -1):
        if x % d == 0:
            return d
    return 1


class TemporalAttentionEncoder(nn.Module):
    """
    Attention-based temporal encoder над последовательностью (B,L,H_in) -> (B,d_model).

    CFG["temporal_mode"]:
      - "xformer_cls": TransformerEncoder + CLS token -> берём CLS
      - "attn_pool":   learnable query + MultiHeadAttention pooling -> берём pooled
    """
    def __init__(
        self,
        in_dim: int,
        d_model: int,
        max_len: int,
        mode: str = "xformer_cls",
        n_heads: int = 4,
        n_layers: int = 1,
        ff_mult: int = 4,
        dropout: float = 0.1,
        use_pos_emb: bool = True,
        causal: bool = False,
    ):
        super().__init__()
        self.mode = mode
        self.max_len = int(max_len)
        self.use_pos_emb = bool(use_pos_emb)
        self.causal = bool(causal)

        self.in_proj = nn.Linear(in_dim, d_model) if in_dim != d_model else nn.Identity()
        self.ln_in = nn.LayerNorm(d_model)
        self.drop = nn.Dropout(dropout)

        # безопасно подгоним heads под d_model
        n_heads = _greatest_divisor_leq(d_model, int(n_heads))
        self.n_heads = n_heads
        self.d_model = d_model

        # positional embeddings
        # для режима CLS нужно +1 позиция под CLS
        pe_len = self.max_len + (1 if mode == "xformer_cls" else 0)
        self.pos_emb = nn.Parameter(torch.zeros(pe_len, d_model)) if self.use_pos_emb else None

        if mode == "xformer_cls":
            self.cls = nn.Parameter(torch.zeros(1, 1, d_model))

            enc_layer = nn.TransformerEncoderLayer(
                d_model=d_model,
                nhead=n_heads,
                dim_feedforward=int(ff_mult * d_model),
                dropout=dropout,
                activation="gelu",
                batch_first=True,
                norm_first=True,
            )
            self.encoder = nn.TransformerEncoder(enc_layer, num_layers=int(max(1, n_layers)))
            self.ln_out = nn.LayerNorm(d_model)

        elif mode == "attn_pool":
            # learnable query token
            self.q = nn.Parameter(torch.zeros(1, 1, d_model))
            self.mha = nn.MultiheadAttention(
                embed_dim=d_model,
                num_heads=n_heads,
                dropout=dropout,
                batch_first=True,
            )
            self.ln_q = nn.LayerNorm(d_model)
            self.ff = nn.Sequential(
                nn.Linear(d_model, 2*d_model),
                nn.GELU(),
                nn.Dropout(dropout),
                nn.Linear(2*d_model, d_model),
            )
            self.ln_out = nn.LayerNorm(d_model)

        else:
            raise ValueError(f"Unknown temporal_mode={mode}. Use 'xformer_cls' or 'attn_pool'.")

        # init
        nn.init.normal_(self.pos_emb, std=0.02) if self.pos_emb is not None else None
        for p in [getattr(self, "cls", None), getattr(self, "q", None)]:
            if isinstance(p, torch.nn.Parameter):
                nn.init.normal_(p, std=0.02)

    def _causal_mask(self, L: int, device):
        # True = masked (не смотреть в будущее)
        m = torch.triu(torch.ones(L, L, device=device, dtype=torch.bool), diagonal=1)
        return m

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        x: (B,L,in_dim)
        return: (B,d_model)
        """
        B, L, _ = x.shape
        if L > self.max_len:
            raise ValueError(f"Sequence length L={L} > max_len={self.max_len}. Increase CFG['lookback'].")

        h = self.in_proj(x)            # (B,L,D)
        h = self.ln_in(h)
        h = torch.nan_to_num(h, nan=0.0, posinf=0.0, neginf=0.0)

        if self.mode == "xformer_cls":
            cls = self.cls.expand(B, 1, self.d_model)  # (B,1,D)
            tokens = torch.cat([cls, h], dim=1)        # (B,1+L,D)

            if self.use_pos_emb:
                # позиции: 0..L (0=CLS, далее 1..L)
                pe = self.pos_emb[:(L+1), :].unsqueeze(0)  # (1,1+L,D)
                tokens = tokens + pe

            tokens = self.drop(tokens)

            # causal mask на tokens с CLS обычно не нужен; если включён — применим только для (1+L)
            src_mask = None
            if self.causal:
                src_mask = self._causal_mask(L+1, device=tokens.device)

            z = self.encoder(tokens, mask=src_mask)  # (B,1+L,D)
            out = self.ln_out(z[:, 0, :])            # CLS
            return torch.nan_to_num(out, nan=0.0, posinf=0.0, neginf=0.0)

        # mode == "attn_pool"
        tokens = h
        if self.use_pos_emb:
            pe = self.pos_emb[:L, :].unsqueeze(0)   # (1,L,D)
            tokens = tokens + pe
        tokens = self.drop(tokens)

        q = self.q.expand(B, 1, self.d_model)        # (B,1,D)
        q = self.ln_q(q)

        # pooled attention: query attends over time tokens
        attn_out, _ = self.mha(query=q, key=tokens, value=tokens, need_weights=False)  # (B,1,D)
        # residual + FFN
        y = q + self.drop(attn_out)
        y2 = self.ff(y)
        y = self.ln_out(y + self.drop(y2))
        out = y[:, 0, :]                              # (B,D)
        return torch.nan_to_num(out, nan=0.0, posinf=0.0, neginf=0.0)


class GNN_Attn_Classifier(nn.Module):
    """
    DROP-IN replacement: имя оставлено тем же, но Attn заменён на Attention temporal encoder.

    forward(x,e,edge_index) -> logits (B,2)
    """
    def __init__(self, node_in, edge_dim, hidden, gnn_layers, Attn_hidden, Attn_layers,
                 dropout=0.1, target_node=2, n_classes=2):
        super().__init__()
        self.target_node = target_node

        self.gnns = nn.ModuleList()
        for i in range(gnn_layers):
            in_dim = node_in if i == 0 else hidden
            self.gnns.append(EdgeGatedMP(in_dim=in_dim, hidden=hidden, edge_dim=edge_dim, dropout=dropout))

        # --- temporal attention config (берём из CFG; если нет — safe defaults)
        temporal_mode = CFG.get("temporal_mode", "xformer_cls")
        attn_heads = CFG.get("attn_heads", 4)
        ff_mult = CFG.get("attn_ff_mult", 4)
        attn_dropout = CFG.get("attn_dropout", None)
        attn_dropout = float(dropout if attn_dropout is None else attn_dropout)
        attn_causal = bool(CFG.get("attn_causal", False))
        use_pos = bool(CFG.get("attn_use_pos_emb", True))
        max_len = int(CFG.get("lookback", 128))

        self.temporal = TemporalAttentionEncoder(
            in_dim=hidden,
            d_model=int(Attn_hidden),
            max_len=max_len,
            mode=str(temporal_mode),
            n_heads=int(attn_heads),
            n_layers=int(max(1, Attn_layers)),
            ff_mult=int(ff_mult),
            dropout=attn_dropout,
            use_pos_emb=use_pos,
            causal=attn_causal,
        )

        self.head = nn.Sequential(
            nn.LayerNorm(int(Attn_hidden)),
            nn.Dropout(dropout),
            nn.Linear(int(Attn_hidden), int(Attn_hidden)),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(int(Attn_hidden), n_classes)
        )

        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.zeros_(m.bias)

    def forward(self, x, e, edge_index):
        h = x
        for gnn in self.gnns:
            h = gnn(h, e, edge_index)  # (B,L,N,H)

        h_tgt = h[:, :, self.target_node, :]         # (B,L,H)
        rep = self.temporal(h_tgt)                   # (B,d_model)
        logits = self.head(rep)                      # (B,2)
        return torch.nan_to_num(logits, nan=0.0, posinf=0.0, neginf=0.0)


# ----------------------------
# SMOKE TEST (сразу после классов)
# ----------------------------
def _smoke_test_model():
    B = 4
    L = CFG["lookback"]
    N = 3
    F = 15
    E = EDGE_INDEX.shape[0]
    W = len(CFG["corr_windows"])

    x = torch.randn(B, L, N, F)
    e = torch.randn(B, L, E, W)

    model = GNN_Attn_Classifier(
        node_in=F,
        edge_dim=W,
        hidden=CFG["hidden"],
        gnn_layers=CFG["gnn_layers"],
        Attn_hidden=CFG["Attn_hidden"],   # now d_model
        Attn_layers=CFG["Attn_layers"],   # now attn layers
        dropout=CFG["dropout"],
        target_node=TARGET_NODE,
        n_classes=2
    )

    with torch.no_grad():
        logits = model(x.float(), e.float(), EDGE_INDEX)
    print("Model ready. logits shape:", tuple(logits.shape), "| temporal_mode:", CFG.get("temporal_mode"))

_smoke_test_model()


Model ready. logits shape: (4, 2) | temporal_mode: xformer_cls


/opt/miniconda3/envs/recbole_new_env/lib/python3.11/site-packages/torch/nn/modules/transformer.py:382: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.norm_first was True
  warnings.warn(


## 8. Training/Eval: Stage A (trade) и Stage B (direction)

In [10]:
# ЛОГИЧЕСКИЙ БЛОК: train/eval helpers for two-stage
# ИСПОЛНЕНИЕ БЛОКА:

@torch.no_grad()
def eval_binary(model, loader, loss_fn, y_key: str = "trade"):
    model.eval()
    ys = []
    probs = []
    ers = []
    total_loss = 0.0
    n = 0

    for x, e, y_trade_b, y_dir_b, er in loader:
        x = x.to(DEVICE).float()
        e = e.to(DEVICE).float()
        y = (y_trade_b if y_key == "trade" else y_dir_b).to(DEVICE).long()

        logits = model(x, e, EDGE_INDEX.to(DEVICE))
        loss = loss_fn(logits, y)

        total_loss += loss.item() * y.size(0)
        n += y.size(0)

        p = torch.softmax(logits, dim=-1).detach().cpu().numpy()
        ys.append(y.detach().cpu().numpy())
        probs.append(p)
        ers.append(er.detach().cpu().numpy())

    ys = np.concatenate(ys) if len(ys) else np.array([], dtype=np.int64)
    probs = np.concatenate(probs) if len(probs) else np.zeros((0, 2), dtype=np.float32)
    ers = np.concatenate(ers) if len(ers) else np.array([], dtype=np.float32)

    if len(ys) == 0:
        return np.nan, np.nan, np.nan, np.nan, None, ys, probs, ers

    y_pred = probs.argmax(axis=1)
    acc = accuracy_score(ys, y_pred)
    f1m = f1_score(ys, y_pred, average="macro")
    auc = roc_auc_score(ys, probs[:, 1]) if len(np.unique(ys)) == 2 else np.nan
    cm = confusion_matrix(ys, y_pred)

    return total_loss / max(n, 1), acc, f1m, auc, cm, ys, probs, ers


@torch.no_grad()
def predict_probs_only(model, loader):
    model.eval()
    probs = []
    ers = []
    for x, e, y_trade_b, y_dir_b, er in loader:
        x = x.to(DEVICE).float()
        e = e.to(DEVICE).float()
        logits = model(x, e, EDGE_INDEX.to(DEVICE))
        p = torch.softmax(logits, dim=-1).detach().cpu().numpy()
        probs.append(p)
        ers.append(er.detach().cpu().numpy())
    probs = np.concatenate(probs) if len(probs) else np.zeros((0, 2), dtype=np.float32)
    ers = np.concatenate(ers) if len(ers) else np.array([], dtype=np.float32)
    return probs, ers


def pnl_proxy_grid_max(prob_trade, prob_dir, exit_ret, thr_trade_grid, thr_dir_grid, cost_bps, min_trades: int = 0):
    """
    Возвращает лучший pnl_mean по grid (per-bar), плюс пороги и статистику.
    min_trades используется как фильтр: комбинации, где сделок меньше, пропускаются.
    Если ни одна комбинация не прошла min_trades — вернём best без фильтра (но это будет fallback-сценарий).
    """
    p_trade = prob_trade[:, 1]
    p_up = prob_dir[:, 1]
    p_dn = 1.0 - p_up
    conf_dir = np.maximum(p_up, p_dn)

    sign = np.where(p_up >= 0.5, 1.0, -1.0).astype(np.float32)
    cost = float(cost_bps) * 1e-4
    N = len(exit_ret)

    best = {
        "pnl_mean": -1e18,
        "pnl_sum": -1e18,
        "thr_trade": None,
        "thr_dir": None,
        "n_trades": 0,
        "trade_rate": 0.0,
        "min_trades_used": int(min_trades),
        "passed_min_trades": False,
    }

    # 1) строгий проход (>=min_trades)
    for thr_t in thr_trade_grid:
        mt = (p_trade >= thr_t)
        for thr_d in thr_dir_grid:
            mask = mt & (conf_dir >= thr_d)
            n_tr = int(mask.sum())
            if n_tr < int(min_trades):
                continue

            pnl = (sign * exit_ret) * mask.astype(np.float32) - cost * mask.astype(np.float32)
            pnl_sum = float(pnl.sum())
            pnl_mean = float(pnl.mean()) if N > 0 else np.nan

            if pnl_mean > best["pnl_mean"]:
                best.update({
                    "pnl_mean": pnl_mean,
                    "pnl_sum": pnl_sum,
                    "thr_trade": float(thr_t),
                    "thr_dir": float(thr_d),
                    "n_trades": n_tr,
                    "trade_rate": float(n_tr / max(1, N)),
                    "passed_min_trades": True,
                })

    # 2) если ничего не прошло min_trades — найдём best без фильтра (для fallback-логов)
    if best["thr_trade"] is None:
        for thr_t in thr_trade_grid:
            mt = (p_trade >= thr_t)
            for thr_d in thr_dir_grid:
                mask = mt & (conf_dir >= thr_d)
                n_tr = int(mask.sum())
                pnl = (sign * exit_ret) * mask.astype(np.float32) - cost * mask.astype(np.float32)
                pnl_sum = float(pnl.sum())
                pnl_mean = float(pnl.mean()) if N > 0 else np.nan

                if pnl_mean > best["pnl_mean"]:
                    best.update({
                        "pnl_mean": pnl_mean,
                        "pnl_sum": pnl_sum,
                        "thr_trade": float(thr_t),
                        "thr_dir": float(thr_d),
                        "n_trades": n_tr,
                        "trade_rate": float(n_tr / max(1, N)),
                        "passed_min_trades": False,
                    })

    return best


def train_binary_classifier(
    X_scaled, edge_feat,
    y_trade_arr, y_dir_arr,
    exit_ret, sample_t,
    idx_train, idx_val, idx_test,
    cfg,
    stage_name: str,
    select_metric: str | None = None,        # "va_auc" | "va_f1m" | "va_pnl_max"
    trade_model_for_pnl=None,                # для stage="dir": фиксированная trade-модель
    idx_val_pnl=None,                        # индексы (sample-space) для pnl-proxy, обычно полный idx_val
):
    """
    Градиентами оптимизируем: CrossEntropyLoss.
    Селектор best checkpoint:
      - trade: обычно va_auc
      - dir:   va_pnl_max, но если best trades < proxy_min_trades => fallback на va_auc
              (реализовано как: best_pnl если были эпохи с trades>=min, иначе best_auc)
    """
    if select_metric is None:
        select_metric = "va_auc"
    if select_metric not in ("va_auc", "va_f1m", "va_pnl_max"):
        raise ValueError("select_metric must be one of: 'va_auc', 'va_f1m', 'va_pnl_max'")

    if select_metric == "va_pnl_max":
        if stage_name != "dir":
            raise ValueError("select_metric='va_pnl_max' supported only for stage_name='dir'")
        if trade_model_for_pnl is None or idx_val_pnl is None:
            raise ValueError("For va_pnl_max you must pass trade_model_for_pnl and idx_val_pnl (full val indices).")

    L = cfg["lookback"]

    tr_ds = LobGraphSequenceDataset2Stage(X_scaled, edge_feat, y_trade_arr, y_dir_arr, exit_ret, sample_t, idx_train, L)
    va_ds = LobGraphSequenceDataset2Stage(X_scaled, edge_feat, y_trade_arr, y_dir_arr, exit_ret, sample_t, idx_val,   L)
    te_ds = LobGraphSequenceDataset2Stage(X_scaled, edge_feat, y_trade_arr, y_dir_arr, exit_ret, sample_t, idx_test,  L)

    tr_loader = DataLoader(tr_ds, batch_size=cfg["batch_size"], shuffle=True,  drop_last=True, collate_fn=collate_fn_2stage)
    va_loader = DataLoader(va_ds, batch_size=cfg["batch_size"], shuffle=False, collate_fn=collate_fn_2stage)
    te_loader = DataLoader(te_ds, batch_size=cfg["batch_size"], shuffle=False, collate_fn=collate_fn_2stage)

    va_pnl_loader = None
    if stage_name == "dir" and (idx_val_pnl is not None):
        va_pnl_ds = LobGraphSequenceDataset2Stage(X_scaled, edge_feat, y_trade_arr, y_dir_arr, exit_ret, sample_t, idx_val_pnl, L)
        va_pnl_loader = DataLoader(va_pnl_ds, batch_size=cfg["batch_size"], shuffle=False, collate_fn=collate_fn_2stage)

    node_in = X_scaled.shape[-1]
    edge_dim = edge_feat.shape[-1]
    model = GNN_Attn_Classifier(
        node_in=node_in, edge_dim=edge_dim,
        hidden=cfg["hidden"], gnn_layers=cfg["gnn_layers"],
        Attn_hidden=cfg["Attn_hidden"], Attn_layers=cfg["Attn_layers"],
        dropout=cfg["dropout"], target_node=TARGET_NODE, n_classes=2
    ).to(DEVICE)


    loss_fn = nn.CrossEntropyLoss()
    opt = torch.optim.AdamW(model.parameters(), lr=cfg["lr"], weight_decay=cfg["weight_decay"])
    sch = torch.optim.lr_scheduler.ReduceLROnPlateau(opt, mode="max", factor=0.5, patience=4)
    scaler_amp = torch.amp.GradScaler('cuda', enabled=(cfg["use_amp"] and DEVICE.type == "cuda"))

    # --- trade-prob на полном val для PnL proxy (считаем 1 раз)
    prob_trade_val_pnl = None
    if stage_name == "dir" and (trade_model_for_pnl is not None) and (va_pnl_loader is not None):
        prob_trade_val_pnl, _ = predict_probs_only(trade_model_for_pnl, va_pnl_loader)

    thr_trade_grid_proxy = cfg.get("proxy_thr_trade_grid") or cfg.get("thr_trade_grid", [0.5])
    thr_dir_grid_proxy   = cfg.get("proxy_thr_dir_grid")   or cfg.get("thr_dir_grid",   [0.5])
    proxy_min_trades = int(cfg.get("proxy_min_trades", 0))

    # --- best trackers
    best_score = -1e18
    best_state = None
    best_epoch = -1
    best_used = select_metric

    # специальные трекеры для va_pnl_max с fallback
    best_score_auc = -1e18
    best_state_auc = None
    best_epoch_auc = -1

    best_score_pnl = -1e18
    best_state_pnl = None
    best_epoch_pnl = -1

    seen_pnl_ok = False

    patience = 8
    bad = 0

    hist = {
        "tr_loss": [], "va_loss": [],
        "va_f1m": [], "va_auc": [],
        "va_pnl_max": [],
        "va_pnl_thr_trade": [],
        "va_pnl_thr_dir": [],
        "va_pnl_n_trades": [],
        "va_sel": [],
        "va_sel_mode": []
    }

    for ep in range(1, cfg["epochs"] + 1):
        # ---- TRAIN
        model.train()
        tot = 0.0
        n = 0

        for x, e, y_trade_b, y_dir_b, er in tr_loader:
            x = x.to(DEVICE).float()
            e = e.to(DEVICE).float()
            y = (y_trade_b if stage_name == "trade" else y_dir_b).to(DEVICE).long()

            opt.zero_grad(set_to_none=True)
            with torch.amp.autocast('cuda', enabled=(cfg["use_amp"] and DEVICE.type == "cuda")):
                logits = model(x, e, EDGE_INDEX.to(DEVICE))
                loss = loss_fn(logits, y)

            if not torch.isfinite(loss):
                continue

            scaler_amp.scale(loss).backward()
            scaler_amp.unscale_(opt)
            nn.utils.clip_grad_norm_(model.parameters(), cfg["grad_clip"])
            scaler_amp.step(opt)
            scaler_amp.update()

            tot += loss.item() * y.size(0)
            n += y.size(0)

        tr_loss = tot / max(n, 1)

        # ---- VAL classification metrics
        va_loss, va_acc, va_f1m, va_auc, va_cm, va_y, va_prob, va_er = eval_binary(
            model, va_loader, loss_fn, y_key=stage_name
        )

        # ---- VAL PnL proxy (dir only)
        va_pnl_best = {"pnl_mean": np.nan, "thr_trade": np.nan, "thr_dir": np.nan, "n_trades": 0, "trade_rate": np.nan,
                       "passed_min_trades": False, "min_trades_used": proxy_min_trades}

        if stage_name == "dir" and (prob_trade_val_pnl is not None) and (va_pnl_loader is not None):
            prob_dir_val_pnl, er_dir_val_pnl = predict_probs_only(model, va_pnl_loader)

            va_pnl_best = pnl_proxy_grid_max(
                prob_trade=prob_trade_val_pnl,
                prob_dir=prob_dir_val_pnl,
                exit_ret=er_dir_val_pnl,
                thr_trade_grid=thr_trade_grid_proxy,
                thr_dir_grid=thr_dir_grid_proxy,
                cost_bps=cfg["cost_bps"],
                min_trades=proxy_min_trades,
            )

        # ---- selection
        sel_val = np.nan
        sel_mode = select_metric

        if select_metric in ("va_auc", "va_f1m"):
            sel_val = (va_auc if select_metric == "va_auc" else va_f1m)
            if not np.isfinite(sel_val):
                sel_val = -1e18

            # единый best
            prev_best = best_score
            if sel_val > best_score:
                best_score = sel_val
                best_epoch = ep
                best_state = {k: v.detach().cpu().clone() for k, v in model.state_dict().items()}
                bad = 0
            else:
                bad += 1

        else:
            # select_metric == "va_pnl_max" (dir only) with hard fallback
            pnl_mean = float(va_pnl_best["pnl_mean"])
            n_tr = int(va_pnl_best["n_trades"])
            pnl_ok = (np.isfinite(pnl_mean) and (n_tr >= proxy_min_trades))

            # обновим best_auc (fallback) всегда
            if np.isfinite(va_auc) and (va_auc > best_score_auc):
                best_score_auc = float(va_auc)
                best_epoch_auc = ep
                best_state_auc = {k: v.detach().cpu().clone() for k, v in model.state_dict().items()}

            # обновим best_pnl только если pnl_ok
            if pnl_ok and (pnl_mean > best_score_pnl):
                best_score_pnl = pnl_mean
                best_epoch_pnl = ep
                best_state_pnl = {k: v.detach().cpu().clone() for k, v in model.state_dict().items()}

            if pnl_ok:
                seen_pnl_ok = True
                sel_val = pnl_mean
                sel_mode = "va_pnl_max"
            else:
                sel_val = float(va_auc) if np.isfinite(va_auc) else -1e18
                sel_mode = f"va_auc_fallback({n_tr}/{proxy_min_trades})"

            # scheduler всегда по текущему sel_val
            if not np.isfinite(sel_val):
                sel_val = -1e18
            sch.step(float(sel_val))

            # early stop: до первой валидной pnl-эпохи -> по AUC, после -> по PnL
            improved = False
            if not seen_pnl_ok:
                # следим за ростом AUC
                improved = (np.isfinite(va_auc) and (float(va_auc) >= best_score_auc))
            else:
                # следим за ростом PnL (только когда pnl_ok)
                improved = pnl_ok and (pnl_mean >= best_score_pnl)

            if improved:
                bad = 0
            else:
                bad += 1

        # если не pnl-метрика — scheduler тут
        if select_metric != "va_pnl_max":
            sch.step(float(sel_val))

        # ---- logging + hist
        hist["tr_loss"].append(tr_loss)
        hist["va_loss"].append(va_loss)
        hist["va_f1m"].append(va_f1m)
        hist["va_auc"].append(va_auc)

        hist["va_pnl_max"].append(float(va_pnl_best["pnl_mean"]) if np.isfinite(va_pnl_best["pnl_mean"]) else np.nan)
        hist["va_pnl_thr_trade"].append(float(va_pnl_best["thr_trade"]) if va_pnl_best["thr_trade"] is not None else np.nan)
        hist["va_pnl_thr_dir"].append(float(va_pnl_best["thr_dir"]) if va_pnl_best["thr_dir"] is not None else np.nan)
        hist["va_pnl_n_trades"].append(int(va_pnl_best["n_trades"]))
        hist["va_sel"].append(float(sel_val) if np.isfinite(sel_val) else np.nan)
        hist["va_sel_mode"].append(sel_mode)

        lr_now = opt.param_groups[0]["lr"]

        # красивый best_str
        if select_metric == "va_pnl_max":
            if best_state_pnl is not None:
                best_str = f"pnl={best_score_pnl:.6f}@ep{best_epoch_pnl:02d}"
            else:
                best_str = f"auc={best_score_auc:.6f}@ep{best_epoch_auc:02d}"
        else:
            best_str = f"{best_score:.6f}@ep{best_epoch:02d}" if best_epoch > 0 else "none"

        if stage_name == "dir":
            print(
                f"[{stage_name}] ep {ep:02d} lr={lr_now:.2e} "
                f"tr_loss={tr_loss:.4f} va_loss={va_loss:.4f} "
                f"f1m={va_f1m:.3f} auc={va_auc:.3f} "
                f"pnl_max={va_pnl_best['pnl_mean']:.6f} "
                f"thr=({va_pnl_best['thr_trade']:.2f},{va_pnl_best['thr_dir']:.2f}) "
                f"trades={va_pnl_best['n_trades']} "
                f"sel({sel_mode})={float(sel_val):.6f} best={best_str}"
            )
        else:
            print(
                f"[{stage_name}] ep {ep:02d} lr={lr_now:.2e} "
                f"tr_loss={tr_loss:.4f} va_loss={va_loss:.4f} "
                f"f1m={va_f1m:.3f} auc={va_auc:.3f} "
                f"sel({select_metric})={float(sel_val):.6f} best={best_str}"
            )

        if bad >= patience:
            break

    # ---- choose final best state
    if select_metric == "va_pnl_max":
        if best_state_pnl is not None:
            model.load_state_dict(best_state_pnl)
            best_score = best_score_pnl
            best_epoch = best_epoch_pnl
            best_used = "va_pnl_max"
        else:
            model.load_state_dict(best_state_auc)
            best_score = best_score_auc
            best_epoch = best_epoch_auc
            best_used = "va_auc_fallback_only"
    else:
        if best_state is not None:
            model.load_state_dict(best_state)
            best_used = select_metric

    # финальные VAL/TEST по best_state
    va_loss, va_acc, va_f1m, va_auc, va_cm, va_y, va_prob, va_er = eval_binary(
        model, va_loader, loss_fn, y_key=stage_name
    )
    te_loss, te_acc, te_f1m, te_auc, te_cm, te_y, te_prob, te_er = eval_binary(
        model, te_loader, loss_fn, y_key=stage_name
    )

    res = {
        "best_val_score": float(best_score),
        "best_epoch": int(best_epoch),
        "select_metric": select_metric,
        "best_used": best_used,

        "val_loss": va_loss,
        "val_acc": va_acc,
        "val_f1m": va_f1m,
        "val_auc": va_auc,
        "val_cm": va_cm,
        "val_y": va_y,
        "val_prob": va_prob,
        "val_er": va_er,

        "test_loss": te_loss,
        "test_acc": te_acc,
        "test_f1m": te_f1m,
        "test_auc": te_auc,
        "test_cm": te_cm,
        "test_y": te_y,
        "test_prob": te_prob,
        "test_er": te_er,

        "hist": hist,
    }
    return model, res



## 9. Two-stage PnL by confidence thresholds

In [11]:
# ЛОГИЧЕСКИЙ БЛОК: PnL по порогам уверенности (two-stage)
# ИСПОЛНЕНИЕ БЛОКА:

def two_stage_pnl_by_threshold(
    prob_trade,          # (N,2) softmax: [:,1]=p_trade
    prob_dir,            # (N,2) softmax: [:,1]=p_up
    exit_ret,            # (N,) realized log-ret to TB exit
    thr_trade: float,
    thr_dir: float,
    cost_bps: float,
):
    p_trade = prob_trade[:,1]
    p_up = prob_dir[:,1]
    p_dn = 1.0 - p_up
    conf_dir = np.maximum(p_up, p_dn)

    trade_mask = (p_trade >= thr_trade) & (conf_dir >= thr_dir)

    action = np.zeros_like(exit_ret, dtype=np.float32)
    action[trade_mask] = np.where(p_up[trade_mask] >= 0.5, 1.0, -1.0)

    cost = (cost_bps * 1e-4) * trade_mask.astype(np.float32)
    pnl = action * exit_ret - cost

    out = {
        "n": len(exit_ret),
        "n_trades": int(trade_mask.sum()),
        "trade_rate": float(trade_mask.mean()),
        "pnl_sum": float(pnl.sum()),
        "pnl_mean": float(pnl.mean()),
        "pnl_sharpe": float((pnl.mean() / (pnl.std() + 1e-12)) * np.sqrt(288)),
    }
    return out

def sweep_thresholds(prob_trade, prob_dir, exit_ret, cfg):
    rows = []
    for thr_t in cfg["thr_trade_grid"]:
        for thr_d in cfg["thr_dir_grid"]:
            m = two_stage_pnl_by_threshold(
                prob_trade=prob_trade,
                prob_dir=prob_dir,
                exit_ret=exit_ret,
                thr_trade=thr_t,
                thr_dir=thr_d,
                cost_bps=cfg["cost_bps"],
            )
            rows.append({"thr_trade":thr_t, "thr_dir":thr_d, **m})
    return pd.DataFrame(rows).sort_values(["pnl_mean","pnl_sum"], ascending=False)

print("Two-stage PnL threshold utils ready.")


Two-stage PnL threshold utils ready.


In [12]:
# ЛОГИЧЕСКИЙ БЛОК: shared helper for probs on arbitrary indices
# ИСПОЛНЕНИЕ БЛОКА:

@torch.no_grad()
def predict_probs_on_indices(model, X_scaled, edge_feat, indices, cfg):
    ds = LobGraphSequenceDataset2Stage(
        X_scaled, edge_feat, y_trade, y_dir, exit_ret, sample_t, indices, cfg["lookback"]
    )
    loader = DataLoader(ds, batch_size=cfg["batch_size"], shuffle=False, collate_fn=collate_fn_2stage)

    model.eval()
    probs = []
    ers = []
    for x, e, yt, yd, er in loader:
        x = x.to(DEVICE).float()
        e = e.to(DEVICE).float()
        logits = model(x, e, EDGE_INDEX.to(DEVICE))
        p = torch.softmax(logits, dim=-1).cpu().numpy()
        probs.append(p)
        ers.append(er.cpu().numpy())

    return np.concatenate(probs), np.concatenate(ers)


## 10. Run folds: scale once → train trade → filter trades → train dir → PnL sweep

In [13]:
# ЛОГИЧЕСКИЙ БЛОК: run walk-forward folds for two-stage training (ONLY on CV-part)
# ИСПОЛНЕНИЕ БЛОКА:

fold_rows = []
models_trade = []
models_dir = []

for fi, (idx_tr, idx_va, idx_te) in enumerate(walk_splits, 1):
    print("\n" + "="*70)
    print(f"FOLD {fi}/{len(walk_splits)} sizes:", len(idx_tr), len(idx_va), len(idx_te))

    # scale once per fold (fit only on train times)
    X_scaled, _ = fit_scale_nodes_train_only(X_node_raw, sample_t, idx_tr, max_abs=CFG["max_abs_feat"])

    # ---- Stage A: trade/no-trade on all samples (по AUC)
    m_trade, r_trade = train_binary_classifier(
        X_scaled, edge_feat,
        y_trade, y_dir,
        exit_ret, sample_t,
        idx_tr, idx_va, idx_te,
        CFG,
        stage_name="trade",
        select_metric="va_auc",
    )
    models_trade.append(m_trade)

    # ---- Stage B: direction ONLY on trade samples (train/val/test индексы фильтруем)
    idx_tr_T = subset_trade_indices(idx_tr, sample_t, y_trade)
    idx_va_T = subset_trade_indices(idx_va, sample_t, y_trade)
    idx_te_T = subset_trade_indices(idx_te, sample_t, y_trade)

    if len(idx_tr_T) < max(200, CFG["batch_size"]*2) or len(idx_te_T) < 50:
        print("[dir] skip: not enough trade samples in this fold.")
        fold_rows.append({
            "fold": fi,
            "trade_test_f1m": r_trade["test_f1m"],
            "dir_test_f1m": np.nan,
            "best_pnl_mean": np.nan,
            "best_thr_trade": np.nan,
            "best_thr_dir": np.nan,
            "n_trades_best": np.nan,
            "trade_rate_best": np.nan,
        })
        continue

    # dir: учим на trade-only, но PnL-proxy считаем на полном idx_va (full val)
    m_dir, r_dir = train_binary_classifier(
        X_scaled, edge_feat,
        y_trade, y_dir,
        exit_ret, sample_t,
        idx_tr_T, idx_va_T, idx_te_T,
        CFG,
        stage_name="dir",
        select_metric="va_pnl_max",
        trade_model_for_pnl=m_trade,
        idx_val_pnl=idx_va,   # <-- полный val для pnl-proxy
    )
    models_dir.append(m_dir)

    # ---- Two-stage PnL evaluation on fold TEST
    prob_trade_te, er_te = predict_probs_on_indices(m_trade, X_scaled, edge_feat, idx_te, CFG)
    prob_dir_te, _       = predict_probs_on_indices(m_dir,   X_scaled, edge_feat, idx_te, CFG)

    sweep = sweep_thresholds(prob_trade_te, prob_dir_te, er_te, CFG)
    best = sweep.iloc[0].to_dict()

    print("PnL on fold-test:",
          "| thr_trade=", best["thr_trade"],
          "| thr_dir=", best["thr_dir"],
          "| pnl_mean=", best["pnl_mean"],
          "| trades=", best["n_trades"])

    fold_rows.append({
        "fold": fi,
        "trade_test_f1m": r_trade["test_f1m"],
        "dir_test_f1m": r_dir["test_f1m"],
        "best_pnl_mean": best["pnl_mean"],
        "best_thr_trade": best["thr_trade"],
        "best_thr_dir": best["thr_dir"],
        "n_trades_best": best["n_trades"],
        "trade_rate_best": best["trade_rate"],
    })

summary = pd.DataFrame(fold_rows)
display(summary)
print("\nMEAN (fold-test внутри CV-part):")
print(summary.mean(numeric_only=True))



FOLD 1/4 sizes: 1168 233 233


/opt/miniconda3/envs/recbole_new_env/lib/python3.11/site-packages/torch/nn/modules/transformer.py:382: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.norm_first was True
  warnings.warn(


[trade] ep 01 lr=2.00e-04 tr_loss=0.9270 va_loss=0.6741 f1m=0.582 auc=0.619 sel(va_auc)=0.619270 best=0.619270@ep01
[trade] ep 02 lr=2.00e-04 tr_loss=0.8142 va_loss=0.6591 f1m=0.380 auc=0.631 sel(va_auc)=0.631002 best=0.631002@ep02
[trade] ep 03 lr=2.00e-04 tr_loss=0.7785 va_loss=0.6602 f1m=0.380 auc=0.612 sel(va_auc)=0.612354 best=0.631002@ep02
[trade] ep 04 lr=2.00e-04 tr_loss=0.7595 va_loss=0.6638 f1m=0.380 auc=0.594 sel(va_auc)=0.593784 best=0.631002@ep02
[trade] ep 05 lr=2.00e-04 tr_loss=0.7323 va_loss=0.6643 f1m=0.380 auc=0.593 sel(va_auc)=0.592852 best=0.631002@ep02
[trade] ep 06 lr=2.00e-04 tr_loss=0.7306 va_loss=0.6681 f1m=0.380 auc=0.601 sel(va_auc)=0.601166 best=0.631002@ep02
[trade] ep 07 lr=1.00e-04 tr_loss=0.7062 va_loss=0.6689 f1m=0.380 auc=0.617 sel(va_auc)=0.616861 best=0.631002@ep02
[trade] ep 08 lr=1.00e-04 tr_loss=0.7203 va_loss=0.6686 f1m=0.380 auc=0.619 sel(va_auc)=0.618803 best=0.631002@ep02
[trade] ep 09 lr=1.00e-04 tr_loss=0.6930 va_loss=0.6687 f1m=0.380 auc=0.

/opt/miniconda3/envs/recbole_new_env/lib/python3.11/site-packages/torch/nn/modules/transformer.py:382: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.norm_first was True
  warnings.warn(


[dir] ep 01 lr=2.00e-04 tr_loss=1.0085 va_loss=0.7215 f1m=0.495 auc=0.473 pnl_max=0.000000 thr=(0.50,0.50) trades=0 sel(va_auc_fallback(0/20))=0.472500 best=auc=0.472500@ep01
[dir] ep 02 lr=2.00e-04 tr_loss=0.8533 va_loss=0.7343 f1m=0.308 auc=0.485 pnl_max=0.000000 thr=(0.50,0.50) trades=0 sel(va_auc_fallback(0/20))=0.485000 best=auc=0.485000@ep02
[dir] ep 03 lr=2.00e-04 tr_loss=0.8765 va_loss=0.6996 f1m=0.505 auc=0.465 pnl_max=0.000000 thr=(0.50,0.50) trades=0 sel(va_auc_fallback(0/20))=0.464500 best=auc=0.485000@ep02
[dir] ep 04 lr=2.00e-04 tr_loss=0.9021 va_loss=0.6937 f1m=0.357 auc=0.451 pnl_max=0.000000 thr=(0.50,0.50) trades=0 sel(va_auc_fallback(0/20))=0.450500 best=auc=0.485000@ep02
[dir] ep 05 lr=2.00e-04 tr_loss=0.8120 va_loss=0.6923 f1m=0.357 auc=0.428 pnl_max=0.000000 thr=(0.50,0.50) trades=0 sel(va_auc_fallback(0/20))=0.428500 best=auc=0.485000@ep02
[dir] ep 06 lr=2.00e-04 tr_loss=0.8145 va_loss=0.6933 f1m=0.422 auc=0.429 pnl_max=0.000000 thr=(0.50,0.50) trades=0 sel(va_au

...

[trade] ep 01 lr=2.00e-04 tr_loss=0.9118 va_loss=0.6448 f1m=0.463 auc=0.555 sel(va_auc)=0.554571 best=0.554571@ep01
[trade] ep 02 lr=2.00e-04 tr_loss=0.7966 va_loss=0.7624 f1m=0.121 auc=0.606 sel(va_auc)=0.605877 best=0.605877@ep02
[trade] ep 03 lr=2.00e-04 tr_loss=0.7559 va_loss=0.7248 f1m=0.198 auc=0.646 sel(va_auc)=0.645522 best=0.645522@ep03
[trade] ep 04 lr=2.00e-04 tr_loss=0.7381 va_loss=0.7317 f1m=0.183 auc=0.643 sel(va_auc)=0.643346 best=0.645522@ep03
[trade] ep 05 lr=2.00e-04 tr_loss=0.7134 va_loss=0.7052 f1m=0.345 auc=0.657 sel(va_auc)=0.657027 best=0.657027@ep05
[trade] ep 06 lr=2.00e-04 tr_loss=0.7219 va_loss=0.6691 f1m=0.551 auc=0.685 sel(va_auc)=0.685323 best=0.685323@ep06
[trade] ep 07 lr=2.00e-04 tr_loss=0.6861 va_loss=0.6603 f1m=0.561 auc=0.694 sel(va_auc)=0.694030 best=0.694030@ep07
[trade] ep 08 lr=2.00e-04 tr_loss=0.6911 va_loss=0.5537 f1m=0.463 auc=0.700 sel(va_auc)=0.699938 best=0.699938@ep08
[trade] ep 09 lr=2.00e-04 tr_loss=0.6870 va_loss=0.4689 f1m=0.463 auc=0.

/opt/miniconda3/envs/recbole_new_env/lib/python3.11/site-packages/torch/nn/modules/transformer.py:382: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.norm_first was True
  warnings.warn(


[dir] ep 01 lr=2.00e-04 tr_loss=0.9663 va_loss=0.6782 f1m=0.499 auc=0.612 pnl_max=0.001279 thr=(0.65,0.55) trades=110 sel(va_pnl_max)=0.001279 best=pnl=0.001279@ep01
[dir] ep 02 lr=2.00e-04 tr_loss=0.8241 va_loss=0.6949 f1m=0.345 auc=0.610 pnl_max=0.000922 thr=(0.50,0.60) trades=128 sel(va_pnl_max)=0.000922 best=pnl=0.001279@ep01
[dir] ep 03 lr=2.00e-04 tr_loss=0.8032 va_loss=0.6998 f1m=0.345 auc=0.594 pnl_max=0.000795 thr=(0.50,0.60) trades=134 sel(va_pnl_max)=0.000795 best=pnl=0.001279@ep01
[dir] ep 04 lr=2.00e-04 tr_loss=0.7900 va_loss=0.6907 f1m=0.405 auc=0.589 pnl_max=0.000785 thr=(0.65,0.55) trades=78 sel(va_pnl_max)=0.000785 best=pnl=0.001279@ep01
[dir] ep 05 lr=2.00e-04 tr_loss=0.7904 va_loss=0.7096 f1m=0.329 auc=0.579 pnl_max=-0.000170 thr=(0.50,0.60) trades=26 sel(va_pnl_max)=-0.000170 best=pnl=0.001279@ep01
[dir] ep 06 lr=1.00e-04 tr_loss=0.7657 va_loss=0.7030 f1m=0.321 auc=0.622 pnl_max=-0.000002 thr=(0.65,0.55) trades=165 sel(va_pnl_max)=-0.000002 best=pnl=0.001279@ep01
[d

,fold,trade_test_f1m,dir_test_f1m,best_pnl_mean,best_thr_trade,best_thr_dir,n_trades_best,trade_rate_best
0,1,0.428922,0.344048,0.000000,0.50,0.5,0.0,0.000000
1,2,0.233553,0.446432,0.000000,0.50,0.5,0.0,0.000000
2,3,0.120755,0.356822,0.000000,0.50,0.5,0.0,0.000000
3,4,0.488186,0.506770,0.000377,0.65,0.5,26.0,0.111588



MEAN (fold-test внутри CV-part):
fold               2.500000
trade_test_f1m     0.317854
dir_test_f1m       0.413518
best_pnl_mean      0.000094
best_thr_trade     0.537500
best_thr_dir       0.500000
n_trades_best      6.500000
trade_rate_best    0.027897
dtype: float64


## 11. Final test

In [14]:
# ЛОГИЧЕСКИЙ БЛОК: Final train on CV(90%) and evaluate once on FINAL(10%)
# ИСПОЛНЕНИЕ БЛОКА:

print("\n" + "="*70)
print("FINAL TRAIN/TEST (CV=90% | FINAL=10%)")

# 1) final train/val split внутри CV-part (по времени)
val_w_final = max(1, int(CFG["val_window_frac"] * n_samples_cv))
train_end = n_samples_cv - val_w_final

idx_train_final = np.arange(0, train_end, dtype=np.int64)
idx_val_final   = np.arange(train_end, n_samples_cv, dtype=np.int64)
idx_test_final  = idx_final_test.astype(np.int64)  # финальный holdout

print("Final split sizes:")
print("  train_final:", len(idx_train_final))
print("  val_final  :", len(idx_val_final))
print("  FINAL test :", len(idx_test_final))

# 2) scaling (fit only on train_final)
X_scaled_final, _ = fit_scale_nodes_train_only(X_node_raw, sample_t, idx_train_final, max_abs=CFG["max_abs_feat"])

# 6) финальная оценка на holdout (БЕЗ подбора порогов на holdout)
prob_trade_hold, er_hold = predict_probs_on_indices(m_trade, X_scaled_final, edge_feat, idx_test_final, CFG)
prob_dir_hold, _         = predict_probs_on_indices(m_dir,   X_scaled_final, edge_feat, idx_test_final, CFG)

final_metrics = two_stage_pnl_by_threshold(
    prob_trade=prob_trade_hold,
    prob_dir=prob_dir_hold,
    exit_ret=er_hold,
    thr_trade=summary['best_thr_trade'][3],
    thr_dir=summary['best_thr_dir'][3],
    cost_bps=CFG["cost_bps"],
)

print("\nFINAL HOLDOUT RESULT (fixed thresholds from val_final):")
print("  pnl_mean :", final_metrics["pnl_mean"])
print("  pnl_sum  :", final_metrics["pnl_sum"])
print("  n_trades :", final_metrics["n_trades"])
print("  trade_rate:", final_metrics["trade_rate"])
print("  sharpe (per-bar proxy):", final_metrics["pnl_sharpe"])

# (опционально) oracle на holdout — НЕ для выбора, только “потолок”
sweep_hold_oracle = sweep_thresholds(prob_trade_hold, prob_dir_hold, er_hold, CFG)
best_hold_oracle = sweep_hold_oracle.iloc[0].to_dict()
print("\n[ORACLE] best possible on holdout by sweeping thresholds (DO NOT USE for selection):")
print("  thr_trade:", best_hold_oracle["thr_trade"], "thr_dir:", best_hold_oracle["thr_dir"])
print("  pnl_mean :", best_hold_oracle["pnl_mean"], "trades:", best_hold_oracle["n_trades"])



FINAL TRAIN/TEST (CV=90% | FINAL=10%)
Final split sizes:
  train_final: 2104
  val_final  : 233
  FINAL test : 260

FINAL HOLDOUT RESULT (fixed thresholds from val_final):
  pnl_mean : -0.0011621993035078049
  pnl_sum  : -0.30217182636260986
  n_trades : 168
  trade_rate: 0.6461538461538462
  sharpe (per-bar proxy): -2.972517084561294

[ORACLE] best possible on holdout by sweeping thresholds (DO NOT USE for selection):
  thr_trade: 0.5 thr_dir: 0.6
  pnl_mean : 0.0 trades: 0.0


In [15]:


# 3) train TRADE on train_final, select by AUC on val_final
m_trade_final, r_trade_final = train_binary_classifier(
    X_scaled_final, edge_feat,
    y_trade, y_dir,
    exit_ret, sample_t,
    idx_train_final, idx_val_final, idx_test_final,
    CFG,
    stage_name="trade",
    select_metric="va_auc",
)

# 4) train DIR on trade-only samples (train/val/test filtered),
#    but pnl-proxy computed on full val_final; selector hard-fallback already inside
idx_train_final_T = subset_trade_indices(idx_train_final, sample_t, y_trade)
idx_val_final_T   = subset_trade_indices(idx_val_final,   sample_t, y_trade)
idx_test_final_T  = subset_trade_indices(idx_test_final,  sample_t, y_trade)

print("Trade-only sizes for DIR:")
print("  train_final_T:", len(idx_train_final_T))
print("  val_final_T  :", len(idx_val_final_T))
print("  test_final_T :", len(idx_test_final_T))

m_dir_final, r_dir_final = train_binary_classifier(
    X_scaled_final, edge_feat,
    y_trade, y_dir,
    exit_ret, sample_t,
    idx_train_final_T, idx_val_final_T, idx_test_final_T,
    CFG,
    stage_name="dir",
    select_metric="va_pnl_max",
    trade_model_for_pnl=m_trade_final,
    idx_val_pnl=idx_val_final,   # pnl-proxy на полном val_final
)

# 5) выбрать пороги по val_final (grid sweep)
prob_trade_val, er_val = predict_probs_on_indices(m_trade_final, X_scaled_final, edge_feat, idx_val_final, CFG)
prob_dir_val, _        = predict_probs_on_indices(m_dir_final,   X_scaled_final, edge_feat, idx_val_final, CFG)

sweep_val = sweep_thresholds(prob_trade_val, prob_dir_val, er_val, CFG)
best_val = sweep_val.iloc[0].to_dict()
thr_trade_star = float(best_val["thr_trade"])
thr_dir_star   = float(best_val["thr_dir"])

print("\nChosen thresholds on val_final:")
print("  thr_trade*:", thr_trade_star)
print("  thr_dir*  :", thr_dir_star)
print("  val pnl_mean:", float(best_val["pnl_mean"]), "| val trades:", int(best_val["n_trades"]))

# 6) финальная оценка на holdout (БЕЗ подбора порогов на holdout)
prob_trade_hold, er_hold = predict_probs_on_indices(m_trade_final, X_scaled_final, edge_feat, idx_test_final, CFG)
prob_dir_hold, _         = predict_probs_on_indices(m_dir_final,   X_scaled_final, edge_feat, idx_test_final, CFG)

final_metrics = two_stage_pnl_by_threshold(
    prob_trade=prob_trade_hold,
    prob_dir=prob_dir_hold,
    exit_ret=er_hold,
    thr_trade=thr_trade_star,
    thr_dir=thr_dir_star,
    cost_bps=CFG["cost_bps"],
)

print("\nFINAL HOLDOUT RESULT (fixed thresholds from val_final):")
print("  pnl_mean :", final_metrics["pnl_mean"])
print("  pnl_sum  :", final_metrics["pnl_sum"])
print("  n_trades :", final_metrics["n_trades"])
print("  trade_rate:", final_metrics["trade_rate"])
print("  sharpe (per-bar proxy):", final_metrics["pnl_sharpe"])

# (опционально) oracle на holdout — НЕ для выбора, только “потолок”
sweep_hold_oracle = sweep_thresholds(prob_trade_hold, prob_dir_hold, er_hold, CFG)
best_hold_oracle = sweep_hold_oracle.iloc[0].to_dict()
print("\n[ORACLE] best possible on holdout by sweeping thresholds (DO NOT USE for selection):")
print("  thr_trade:", best_hold_oracle["thr_trade"], "thr_dir:", best_hold_oracle["thr_dir"])
print("  pnl_mean :", best_hold_oracle["pnl_mean"], "trades:", best_hold_oracle["n_trades"])


/opt/miniconda3/envs/recbole_new_env/lib/python3.11/site-packages/torch/nn/modules/transformer.py:382: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.norm_first was True
  warnings.warn(


[trade] ep 01 lr=2.00e-04 tr_loss=0.8149 va_loss=0.6645 f1m=0.467 auc=0.557 sel(va_auc)=0.556643 best=0.556643@ep01
[trade] ep 02 lr=2.00e-04 tr_loss=0.7353 va_loss=0.6609 f1m=0.380 auc=0.564 sel(va_auc)=0.563947 best=0.563947@ep02
[trade] ep 03 lr=2.00e-04 tr_loss=0.7083 va_loss=0.6686 f1m=0.473 auc=0.542 sel(va_auc)=0.541880 best=0.563947@ep02
[trade] ep 04 lr=2.00e-04 tr_loss=0.6835 va_loss=0.6794 f1m=0.561 auc=0.538 sel(va_auc)=0.538306 best=0.563947@ep02
[trade] ep 05 lr=2.00e-04 tr_loss=0.6729 va_loss=0.7333 f1m=0.498 auc=0.541 sel(va_auc)=0.540793 best=0.563947@ep02
[trade] ep 06 lr=2.00e-04 tr_loss=0.6558 va_loss=0.6957 f1m=0.558 auc=0.543 sel(va_auc)=0.543434 best=0.563947@ep02
[trade] ep 07 lr=1.00e-04 tr_loss=0.6491 va_loss=0.7600 f1m=0.529 auc=0.557 sel(va_auc)=0.557265 best=0.563947@ep02
[trade] ep 08 lr=1.00e-04 tr_loss=0.6396 va_loss=0.7102 f1m=0.574 auc=0.564 sel(va_auc)=0.564258 best=0.564258@ep08
[trade] ep 09 lr=1.00e-04 tr_loss=0.6479 va_loss=0.7289 f1m=0.556 auc=0.

/opt/miniconda3/envs/recbole_new_env/lib/python3.11/site-packages/torch/nn/modules/transformer.py:382: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.norm_first was True
  warnings.warn(


[dir] ep 01 lr=2.00e-04 tr_loss=0.9104 va_loss=0.6827 f1m=0.344 auc=0.700 pnl_max=-0.000011 thr=(0.70,0.55) trades=25 sel(va_pnl_max)=-0.000011 best=pnl=-0.000011@ep01
[dir] ep 02 lr=2.00e-04 tr_loss=0.8443 va_loss=0.6966 f1m=0.350 auc=0.492 pnl_max=0.000988 thr=(0.65,0.55) trades=61 sel(va_pnl_max)=0.000988 best=pnl=0.000988@ep02
[dir] ep 03 lr=2.00e-04 tr_loss=0.7795 va_loss=0.6902 f1m=0.467 auc=0.513 pnl_max=0.001205 thr=(0.65,0.50) trades=97 sel(va_pnl_max)=0.001205 best=pnl=0.001205@ep03
[dir] ep 04 lr=2.00e-04 tr_loss=0.7545 va_loss=0.6878 f1m=0.536 auc=0.532 pnl_max=0.000883 thr=(0.65,0.50) trades=97 sel(va_pnl_max)=0.000883 best=pnl=0.001205@ep03
[dir] ep 05 lr=2.00e-04 tr_loss=0.7543 va_loss=0.6904 f1m=0.350 auc=0.520 pnl_max=0.000885 thr=(0.65,0.50) trades=97 sel(va_pnl_max)=0.000885 best=pnl=0.001205@ep03
[dir] ep 06 lr=2.00e-04 tr_loss=0.7385 va_loss=0.6924 f1m=0.350 auc=0.531 pnl_max=0.001262 thr=(0.65,0.55) trades=85 sel(va_pnl_max)=0.001262 best=pnl=0.001262@ep06
[dir] e